# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [49]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

g_key = g_key[:-1]

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [73]:
cities_df = pd.read_csv("../output_data/cities.csv", keep_default_na=False)
del cities_df['Unnamed: 0']
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,severo-kurilsk,50.68,156.12,49.39,62.0,9.0,10.36,RU,06/27/20
1,bredasdorp,-34.53,20.04,59.00,87.0,53.0,21.92,ZA,06/27/20
2,albany,42.60,-73.97,66.00,74.0,95.0,3.89,US,06/27/20
3,rikitea,-23.12,-134.97,72.21,79.0,15.0,13.18,PF,06/27/20
4,kruisfontein,-34.00,24.73,67.73,31.0,0.0,15.21,ZA,06/27/20


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [108]:
# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]

# Set humidity as weight
humidity = cities_df["Humidity"].astype(float)


# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False,
                                 max_intensity=100,
                                 point_radius=2)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [82]:
ideal_cities = cities_df.loc[(cities_df["Humidity"]< 50)&
                             (cities_df["Cloudiness"] < 20.0) &
                             (cities_df["Max Temp"] > 60.0) & 
                              (cities_df["Max Temp"] < 90.0),:]

print(f"There are {len(ideal_cities)} cities that fit weather criteria. There are no null values.")

There are 35 cities that fit weather criteria. There are no null values.


In [79]:
ideal_cities["Country"].value_counts()

RU    7
ZA    5
NA    4
MA    3
FI    2
CN    2
MG    2
ZM    1
MN    1
NO    1
MZ    1
IR    1
US    1
LY    1
AU    1
UA    1
GR    1
Name: Country, dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [90]:
hotel_df = ideal_cities.copy()
hotel_df['Hotel Name']=''
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
4,kruisfontein,-34.00,24.73,67.73,31.0,0.0,15.21,ZA,06/27/20,
23,port alfred,-33.59,26.89,74.44,25.0,0.0,23.51,ZA,06/27/20,
25,port elizabeth,-33.92,25.57,69.80,25.0,0.0,18.34,ZA,06/27/20,
46,opuwo,-18.06,13.84,77.65,17.0,0.0,12.73,NA,06/27/20,
57,nador,35.17,-2.93,87.80,26.0,0.0,14.99,MA,06/27/20,


In [93]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
    except (TypeError, IndexError):
        print("Missing field/result... skipping.")


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [105]:
# Drop rows with empty strings
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)
hotel_df.dropna(how = 'any',inplace = True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
4,kruisfontein,-34.00,24.73,67.73,31.0,0.0,15.21,ZA,06/27/20,Oyster Bay House Rental
23,port alfred,-33.59,26.89,74.44,25.0,0.0,23.51,ZA,06/27/20,The Halyards Hotel
25,port elizabeth,-33.92,25.57,69.80,25.0,0.0,18.34,ZA,06/27/20,39 On Nile Guest House
46,opuwo,-18.06,13.84,77.65,17.0,0.0,12.73,NA,06/27/20,ABBA Guesthouse
57,nador,35.17,-2.93,87.80,26.0,0.0,14.99,MA,06/27/20,Hôtel La Giralda
60,east london,-33.02,27.91,75.20,29.0,0.0,5.82,ZA,06/27/20,Tu Casa
68,krasnyy oktyabr,56.12,38.88,82.47,41.0,9.0,12.97,RU,06/27/20,Priozernaya Hotel
85,eenhana,-17.47,16.33,80.60,15.0,0.0,8.05,NA,06/27/20,Hotel Litu
87,saint george,37.10,-113.58,75.20,23.0,1.0,6.93,US,06/27/20,Best Western Plus Abbey Inn
99,nikolskoye,59.70,30.79,78.80,39.0,0.0,15.66,RU,06/27/20,Tourist House - Sablino


In [110]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [112]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))